# Configuration

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
import seaborn as sns
import os
from os.path import join
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import OneHotEncoder
# import shap

import xgboost
print(xgboost.__version__)
from xgboost import XGBClassifier

import pickle
import time
import json

!export PYTHONPATH="/home/hendrio/amex-1M/:$PYTHONPATH"
!export PYTHONPATH="/home/hendrio/amex-1M/src/:$PYTHONPATH"


import sys
sys.path.append('/home/hendrio/amex-1M/')
sys.path.append('/home/hendrio/amex-1M/src/')

2.0.3


In [2]:
base_dir = "/home/hendrio/amex-1M/"
os.listdir(base_dir)

paths={
    'data': join(base_dir, 'data'),
    'src': join(base_dir, 'data', 'src'),
    'processed': join(base_dir, 'data', 'processed'),
    'datasets': join(base_dir, 'datasets1'),
    'results': join(base_dir,'results'),
    'images': join(base_dir,'images'),
    
}


# Load Dataset

In [5]:
def load_amex_1m(path, file_name='amex-1M_binary-dataset-[intents-permissions-apicalls].npz'):
    data = np.load(join(path, file_name), allow_pickle=True)
    metadata = dataframe(data['metadata'], columns=data['metadata_columns'])
    
    columns_names = data['column_names']
    sha256 = data['sha256']

    labels_ohe = OneHotEncoder().fit_transform(np.expand_dims(metadata['CLASS'].values, axis=1)).toarray()

    print(data['data'].shape, labels_ohe.shape)

    # if ("CLASS" not in metadata.columns):
    #     print()
    #     metadata['CLASS'] = create_class(df=metadata['VT_SCANNERS_MALICIOUS'], threshold=4)
    
    print(metadata['CLASS'].value_counts())
    return data['data'], labels_ohe, metadata, columns_names, sha256


def create_class(df, threshold):
    return np.asarray([1 if i>=threshold else 0 for i in df ])

In [6]:
dataset_name = 'amex-1M-[intents-permissions-opcodes-apicalls]-chi2'

In [7]:
data, labels_ohe, metadata, columns_names, sha256 = load_amex_1m(
    path=paths['processed'], 
    file_name=f'{dataset_name}.npz'
    )

(1340515, 12409) (1340515, 2)
CLASS
0    1221421
1     119094
Name: count, dtype: int64


# Eval

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.base import clone
from scipy import stats
import joblib
import trainer

def stratified_k_fold_evaluation(model, data, labels_ohe, k=10, path_save=None, dir_name=None):
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=0)
    confusion_matrices, classification_reports = [], []
    folds_report, cm_report = {}, {}
    best_model, best_train_index, best_test_index = None, None, None
    best_score = 0
    fold = 1

    if path_save is not None:
        if dir_name is None:
            raise Exception("Provid a 'dir_name' parameter to save results. ")
        
        os.makedirs(join(path_save, dir_name), exist_ok=True)
        os.makedirs(join(path_save, dir_name, 'KFold'), exist_ok=True)

    for train_index, test_index in skf.split(data, np.argmax(labels_ohe, axis=1)):
        print(f'fold {fold}')
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = labels_ohe[train_index], labels_ohe[test_index]

        # Clone the model to ensure a fresh model for each fold
        model_fold = clone(model)
        model_fold.fit(X_train, y_train)
        y_pred = model_fold.predict(X_test)

        # Calculate confusion matrix and classification report
        cm = confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))
        report = classification_report(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), output_dict=True)
        print(cm)
        print(report)
        confusion_matrices.append(cm)
        classification_reports.append(report)
        folds_report[f'fold-{fold}'] = report
        cm_report[f'fold-{fold}'] = cm.tolist()

        # Calculate accuracy
        accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))

        # Check if the current fold model is the best so far
        if accuracy > best_score:
            best_score = accuracy
            best_model = clone(model_fold)
            best_train_index = train_index
            best_test_index = test_index
        
        fold += 1

    # Save the best model to a file
    os.makedirs(os.path.join(path_save, dir_name), exist_ok=True)
    joblib.dump(best_model, os.path.join(path_save, dir_name, 'XGBClassifier.pkl'))
    print(f"Best model saved with accuracy: {best_score}")

    # Save the train and test indices of the best fold
    np.save(os.path.join(path_save, dir_name, 'best_train_index.npy'), best_train_index)
    np.save(os.path.join(path_save, dir_name, 'best_test_index.npy'), best_test_index)

    with open(join(path_save, dir_name, 'KFold', f'classification_report_folds.json'), 'w') as f:
        json.dump(folds_report, f, indent=4)

    with open(join(path_save, dir_name, 'KFold', f'cm_folds.json'), 'w') as f:
        json.dump(cm_report, f, indent=4)

    # Aggregate confusion matrices and classification reports
    # final_confusion_matrix = np.mean(confusion_matrices, axis=0)
    # Using SUM
    final_confusion_matrix = np.sum(confusion_matrices, axis=0)

    print(f'confusion matrix for fold {fold}')
    print(final_confusion_matrix)
    
    trainer.plot_confusion_matrix(
        final_confusion_matrix, 
        class_labels=['Benign', 'Malware'], 
        path_save=os.path.join(path_save, dir_name, f'{dir_name}-confusion_matrix')
        )

    final_classification_report = {}

    # Calculate mean and confidence intervals for each metric
    for key in classification_reports[0].keys():
        metrics = [report[key] for report in classification_reports]
        if isinstance(metrics[0], dict):  # Handle nested dicts for per-class metrics
            final_classification_report[key] = {}
            for metric in metrics[0].keys():
                
                if metric=="support": ## does not aggregate support information
                    continue

                values = [m[metric] for m in metrics]
                mean = np.mean(values)
                std_dev = np.std(values)
                conf_int = stats.t.interval(0.95, len(values)-1, loc=mean, scale=stats.sem(values))
                margin = conf_int[1] - mean
                final_classification_report[key][metric] = {
                    'mean': mean,
                    'std_dev': std_dev,
                    '95% CI': conf_int,
                    'margin': margin
                }
        else:
            values = metrics
            mean = np.mean(values)
            std_dev = np.std(values)
            conf_int = stats.t.interval(0.95, len(values)-1, loc=mean, scale=stats.sem(values))
            margin = conf_int[1] - mean
            final_classification_report[key] = {
                'mean': mean,
                'std_dev': std_dev,
                '95% CI': conf_int,
                'margin': margin
            }
    if path_save is not None:
        # Save final aggregated results
        pd.DataFrame(final_confusion_matrix).to_csv(join(path_save, dir_name,'final_confusion_matrix.csv'))
        pd.DataFrame(final_classification_report).to_csv(join(path_save, dir_name,'final_classification_report.csv'))
        with open(os.path.join(path_save, f'{dir_name}-report.json'), 'w') as f:
            json.dump(final_classification_report, f, indent=4)

    return final_confusion_matrix, final_classification_report


# Example usage:
# from sklearn.ensemble import RandomForestClassifier
# model = RandomForestClassifier()
# data = np.random.rand(100, 10)  # Example data
# labels_ohe = np.eye(3)[np.random.choice(3, 100)]  # Example one-hot encoded labels
# stratified_k_fold_evaluation(model, data, labels_ohe)


In [9]:
print(dataset_name)

amex-1M-[intents-permissions-opcodes-apicalls]-chi2


In [10]:
print(data.shape, labels_ohe.shape)

(1340515, 12409) (1340515, 2)


In [11]:
stratified_k_fold_evaluation(
    model=XGBClassifier(random_state=0), 
    data=data, 
    labels_ohe=labels_ohe, 
    k=10, 
    path_save=join(paths['results'], 'models'), 
    dir_name=f'{dataset_name}-XGBClassifier-KFold10')

fold 1
[[121523    620]
 [  1390  10519]]
{'0': {'precision': 0.9886911880761189, 'recall': 0.9949239825450497, 'f1-score': 0.9917977931574824, 'support': 122143.0}, '1': {'precision': 0.9443397073345902, 'recall': 0.8832815517675707, 'f1-score': 0.9127906976744186, 'support': 11909.0}, 'accuracy': 0.9850058186375437, 'macro avg': {'precision': 0.9665154477053546, 'recall': 0.9391027671563101, 'f1-score': 0.9522942454159504, 'support': 134052.0}, 'weighted avg': {'precision': 0.9847510619746743, 'recall': 0.9850058186375437, 'f1-score': 0.9847789086939324, 'support': 134052.0}}
fold 2
